In [2]:
from sympy import *
from LieAlgebras5 import *

## VECTOR SPACES

In [4]:
V = VectorSpace()
V.basis_symbolic = ['b1', 'b2']
V.basis_symbolic

[b1, b2]

In [5]:
[b1, b2] = V.basis_symbolic
a1, a2 = symbols('a1 a2')
v_symbolic = 3*a1*b1 + a2*b2
display(v_symbolic)
v_array = V.from_symbols_to_array(v_symbolic)
display(v_array)
print(v_array)

3*a1*b1 + a2*b2

[[3*a1], [a2]]

[[3*a1], [a2]]


In [27]:
v_array[1,0]

a2

In [6]:
V.from_array_to_symbols(v_array) == v_symbolic

True

In [7]:
V.from_array_to_symbols([1,2])

b1 + 2*b2

In [8]:
V.from_array_to_symbols(['a','b'])

a*b1 + b*b2

In [9]:
V.a_vector_symbolic('x') 
# notice that the internal and standard indexing start with 0, 
# while in this notebook we were starting from 1

x_0*b1 + x_1*b2

In [18]:
for b in V.basis_coord:
    display(b)
   

[[1], [0]]

[[0], [1]]

In [19]:
V.basis_outer

ValueError: An outer basis is not set yet.

### Basis of vector spaces

A vector space has three bases, which are three representations of the same basis:
1. A symbolic basis `V.basis_symbolic`
2. An array basis `V.basis_coord`
3. An outer basis `V.basis_outer`

The first two are more or less automatic and one can go from one to another using the methods
`V.from_array_to_symbols` and `V.from_symbols_to_array`,
for both elements of the basis or their linear combinations.

The outer basis is instead not set a-priori: in fact, it raises an exeption if called without being set.
Similarly, the methods from or to the outer basis are defined but they don't do anything.
The ratio is the following: the outer basis can be made of objects chosen by the user. 
For instance, they could be monomials, and `V` being a vector space of polynomials.
Or, they could be matrices and `V` being a vector space of matrices.

In [30]:
# Example of use of an outer basis
# (for another example, see the dual space)
m1 = Matrix([[1,0],[0,1]])
m2 = Matrix([[0,-1],[1,0]])
V.basis_outer = [m1,m2]
V.from_array_to_outer = lambda v : v[0,0] * V.basis_outer[0] + v[1,0] * V.basis_outer[1]
V.from_outer_to_array = lambda m : m[0,0] * V.basis_coord[0] + m[1,0] * V.basis_coord[1]
V.from_symbols_to_outer = lambda v : V.from_array_to_outer(V.from_symbols_to_array(v)) 
V.from_outer_to_symbols = lambda m : V.from_array_to_symbols(V.from_outer_to_array(v))

In [29]:
V.from_symbols_to_outer(2*b1 + 3* b2)

Matrix([
[2, -3],
[3,  2]])

### Gradings of vector spaces

A grading is given by a organization of the basis into layers.
Each layer has a positive integer as weight.
Only integer weights 1, 2, ... are implemented.
A grading defines dilations `V.dil`

In [8]:
V.growth_vector = [1,1]
V.growth_vector

[1, 1]

In [9]:
V.graded_basis_symbolic

[[b1], [b2]]

In [10]:
# Notice that, since we had already set a symbolic basis, we still have the same:
V.basis_symbolic

[b1, b2]

In [11]:
# We can, and it is better to do it, uniform the two symbolic basis:
# Notice that we don't have a method to transform expressions in the graded basis
# if we it is not uniformed to the symbolic basis.
V.basis_symbolic = flatten(V.graded_basis_symbolic)
V.basis_symbolic

[b1, b2]

In [12]:
display(V.weights)
display(v_array)
display(V.dil(2,v_array))

[1, 2]

[[3*a1], [a2]]

Matrix([
[6*a1],
[4*a2]])

### Dual space

There is an outomatic construction of dual space
The point about the dual space is that it is in duality with the original space via the method
`V.pairing_dualVSvect_symbolic`.
As the name suggests, it takes in two inputs `(b,a)` that are both symbolic vectors, the first one `b` is a symbolic element of the dual and the second one `a` is a symbolic element of the vector space.

In [13]:
# This is the automatic symbolic dual basis:
V.dual_basis_symbolic

[@b1, @b2]

In [14]:
# If you really want to have different symbols, you can use the inned method that builds it:
V._dual_basis_symbolic_build(pre_symbol_for_dual='(',post_symbol_for_dual=')^*')
V.dual_basis_symbolic

[(b1)^*, (b2)^*]

In [15]:
V_dual = V.dual_space
display( V_dual.basis_symbolic )

[(b1)^*, (b2)^*]

In [16]:
# notice that vector in the dual space are still vertical arrays:
display( V_dual.basis_coord )
# A way to get around this is to define an outer basis of horizontal arrays:
dim = V_dual.dimension
V_dual.basis_outer = [ Array(v, (1,dim)) for v in eye(dim).rowspace()]
display(V_dual.basis_outer)

[[[1], [0]], [[0], [1]]]

[[[1, 0]], [[0, 1]]]

In [17]:
# At this point, we can define the methods to transform things from one to the other:
V_dual.from_array_to_outer = lambda v: v.reshape(1,V_dual.dimension)
V_dual.from_outer_to_array = lambda v: v.reshape(V_dual.dimension,1)
V_dual.from_symbols_to_outer = lambda v: V_dual.from_array_to_outer( V_dual.from_symbols_to_array(v) )
V_dual.from_outer_to_symbols = lambda v: V_dual.from_array_to_symbols( V_dual.from_outer_to_array(v) )

In [18]:
# We have a method for _ordered_ duality of symbolic expressions
for a in V.basis_symbolic :
    for b in V.dual_basis_symbolic:
        print(b,a,V.pairing_dualVSvect_symbolic(b,a) )
    print()

(b1)^* b1 1
(b2)^* b1 0

(b1)^* b2 0
(b2)^* b2 1



## LINEAR MAPS

A linear map is a multilinear map with only one domain.

In [32]:
L = MultLinMap()
L.domains = V,V
L.range = V
L.rules = {}
L(b1,b1)

0

In [39]:
L = MultLinMap()
# A multilinear map `L` has a list of domains `L.domains` and a range `L.range`, all vector spaces:
L.domains = V,V
L.range = V
# The map can be described by rules on tuples of symbolic elements of the basis. 
# The missing pairs are assumed to be zero.
# A multilinear map is callable. The call is just an application of the method L.apply()
[b1, b2] = V.basis_symbolic
L.rules = {}
print('L(b1,b2) = ',L(b1,b2))

L.rules = {(b1,b2): b1, (b2,b1): -b1}
print('L(b1,b2) = ',L(b1,b2))

# L.apply(a*3*b2 , a*b1 + b2)
a = symbols('a')
L(a*3*b2 , a*b1 + b2)

L(b1,b2) =  0
L(b1,b2) =  b1


-3*a**2*b1

In [40]:
# A linear map can read both symbolic and array vectors
L( Array([1,0],(2,1)) , Array([0,1],(2,1)) )

[[1], [0]]

In [42]:
# A linear map is a multilinear map with only one domain.
L = LinMap()
# this snipped works also with 
# L = MultLinMap()
L.domains = V
L.range = V
[b1, b2] = V.basis_symbolic
a = symbols('a')
# if L = LinMap(), then any of the following two lines works fine.
# however, if L = MultLinMap(), then only the second one works 
L.rules = {b1:b2, b2: -b1}
# L.rules = {(b1,):b2, (b2,): -b1}
L(3*a*b1 + b2)

3*a*b2 - b1

In [22]:
L( Array([1,0],(2,1)) )

[[0], [1]]

## LIE ALGEBRAS

A Lie algebra is a vector space endowed with a bilinear map, the Lie brackets.
The class `LieAlgebra` is an extension of `VectorSpace`.

### The Heisenberg algebra

In [3]:
heis = LieAlgebra()
heis.name = 'Heisenberg'
# Notice that `heis` is first of all an instance of the class `VectorSpace`.
# As such, we can use the same objects we had for `V` above.
heis.basis_symbolic = ['X','Y','Z']
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}

In [4]:
# the class heis is callable:
print('heis(X,Y) = ',heis(X,Y))

[e1,e2,e3] = heis.basis_coord
print('heis(e1,e2) = ',heis(e1,e2))

# it calls the method heis.brackets,
# which is actually an instance of MultLinMap
print(type(heis.brackets))
# Notice that the rules are made automatically antisymmetric:
print('heis.brackets.is_antisymmetric = ', heis.brackets.is_antisymmetric)
print('heis.brackets.rules = ', heis.brackets.rules) 
print('heis(X,Y) = ', heis(X,Y))
print('heis(Y,X) = ', heis(Y,X))

heis(X,Y) =  Z
heis(e1,e2) =  [[0], [0], [1]]
<class 'LieAlgebras5.MultLinMap'>
heis.brackets.is_antisymmetric =  True
heis.brackets.rules =  {(X, Y): Z, (Y, X): -Z}
heis(X,Y) =  Z
heis(Y,X) =  -Z


In [5]:
# It is good to check the Jacobi identity when defying a Lie algebra:
heis.check_jacobi()

True

In [6]:
# Brackets have a description as tensor:
display(heis.brackets.as_tensor)
# structure constants are derived from this tensor, which is much larger than necessary
display(heis.structure_constants)
help(heis.__class__.structure_constants)

[[[[[[0, 0, 0]], [[0, 0, 0]], [[0, 0, 0]]]]], [[[[[0, 0, 0]], [[0, 0, 0]], [[0, 0, 0]]]]], [[[[[0, 1, 0]], [[-1, 0, 0]], [[0, 0, 0]]]]]]

[[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 1, 0], [-1, 0, 0], [0, 0, 0]]]

Help on property:

    Returns the structure constants of the Lie algebra,
    
    that is a tensor Gamma of shape (dim,dim,dim) so that
    self(b_i,b_j) = sum_k Gamma[^k,_i,_j] b_k



In [7]:
# Lie algebras can have a grading or a stratification.
# They both correspond to a grading of the underlying vector space.
# `heis` is stratified, and so we declare it as such:
heis.declare_stratified(isit = True, growth_vector = [2,1], step = 2)
# This declaration will set other properties of `heis`, such as the nilpotency step, and simplify several algorithms, for instance for the bch formula.
# Some methods are available only if the lie algebra is graded or stratified.
# One can also separately declare nilpotency or grading.
print('heis.is_stratified = ',heis.is_stratified)
print('heis.is_graded = ',heis.is_graded)
print('heis.is_nilpotent = ',heis.is_nilpotent)
print('heis.step = ',heis.step)

heis.is_stratified =  True
heis.is_graded =  True
heis.is_nilpotent =  True
heis.step =  2


In [11]:
# Notice that at this point the automatic graded basis is recovered from the symbolic one:
display(heis.graded_basis_symbolic)
# If we want a graded basis with the correct indices, we can do this:
heis._graded_basis_symbolic_build(force_build = True)
display(heis.graded_basis_symbolic)
# If heis.basis_symbolic was not defined already, it would be overwritten by now.
# However, since we forced the method, heis.basis_symbolic has not been overwritten.
display(heis.basis_symbolic)
# HOWEVER, THIS CELL SEEM TO BREAK THE CONSTRUCTION OF THE JET SPACE

[[b^0_0, b^0_1], [b^1_0]]

[[b^0_0, b^0_1], [b^1_0]]

[X, Y, Z]

In [12]:
# THIS CELL IS TO FIX THE DAMAGE OF THE PREVIOUS ONE.
heis = LieAlgebra()
heis.name = 'Heisenberg'
heis.basis_symbolic = ['X','Y','Z']
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}
heis.declare_stratified(isit = True, growth_vector = [2,1], step = 2)

In [13]:
# We have an implementation of the BCH formula, which works pretty well when the group is nilpotent.
display(heis.bch(X+Y,Y+Z))
display(heis.bch(e1+e2,e2+e3))

X + 2*Y + 3*Z/2

[[1], [2], [3/2]]

In [31]:
# there is a method that produces some technical data that are useful on Lie algebras:
print(heis._a_basis_of_brackets_build.__doc__ )
# this produces the following three objects (not very readable indeed)
display(heis.a_basis_of_brackets)
display(heis.a_basis_of_brackets_graded)
display(heis.a_basis_of_brackets_dict)


        Builds self._a_basis_of_brackets_graded , self._a_basis_of_brackets , and self._a_basis_of_brackets_dict

        that is, 
        a basis in coordinates for *self*
        out of the set of all brackets of the first layer,
        both organized in layers and flattened,
        and a dictionary that tells you for each element of the basis which are the two vectors it is a bracket of.

        Works only for stratified Lie algebras.
        


[[[1], [0], [0]], [[0], [1], [0]], [[0], [0], [1]]]

[[[[1], [0], [0]], [[0], [1], [0]]], [[[0], [0], [1]]]]

{[[0], [0], [1]]: [[[1], [0], [0]], [[0], [1], [0]]]}

In [32]:
# We also have similar methods that boild everything down to the first layer:
print(heis.__class__.a_basis_symbolic_of_brackets_dict.__doc__)
display(heis.a_basis_symbolic_of_brackets_dict)
print(heis.__class__.a_basis_symbolic_of_brackets_dict_V1.__doc__)
display(heis.a_basis_symbolic_of_brackets_dict_V1)
display(heis.boil_to_V1(Z))


Like self.a_basis_of_brackets_dict(), but using symbolic basis.
        


{Z: X*Y - Y*X}


Returns a dictionary {z: x*y-y*x}

so that z is the bracket x and y, and both x and y are in the first layer only.
So, it is like self.a_basis_symbolic_of_brackets_dict(),
but where only elements of the first layer appear
        


{Z: X*Y - Y*X}

X*Y - Y*X

### SUBALGEBRAS

In [14]:
# This defines the second heisenberg group
heis2 = LieAlgebra()
heis2.name = 'Heisenberg 2'
heis2.basis_symbolic = ['X1','Y1','X2','Y2','Z']
[X1,Y1,X2,Y2,Z] = heis2.basis_symbolic
heis2.brackets.rules = {(X1,Y1):Z, (X2,Y2):Z}
print(heis2.brackets.rules)
heis2.declare_stratified(isit = True, growth_vector = [4,1], step = 2)

{(X1, Y1): Z, (X2, Y2): Z, (Y1, X1): -Z, (Y2, X2): -Z}


In [17]:
# A subalgebra is a new algebra defined using the ambient algebra:
W = heis2.lie_subalgebra([X1,Y1,Z],'a')
A = W.basis_symbolic
print('A = ',A)
print('W(A[0],A[1]) = ',W(A[0],A[1]))

A =  [a_0, a_1, a_2]
W(A[0],A[1]) =  a_2


### MORPHISMS, EMBEDDINGS, ISOMORPHISMS

In [18]:
# This snipped makes this part self-contained.
heis = LieAlgebra()
heis.dimension = 3
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}

heis2 = LieAlgebra()
heis2.dimension = 5
[X1,Y1,X2,Y2,Z] = heis2.basis_symbolic
heis2.brackets.rules = {(X1,Y1):Z, (X2,Y2):Z}

W1 = heis2.lie_subalgebra([X1,Y1,Z],'a')

W2 = heis2.lie_subalgebra([X1,X2,Z],'aa')

In [26]:
# Given two Lie algebras, `heis` and `W1` in this case, 
# we can compute all lie-algebras morphism from the first to the second:
L, sol = heis.morphisms_to(W1)
# `L` is a generic linear map, 
# `sol` is the collection of rules on the coefficients of `L` to make `L` into a Lie algebra morphism.
display(L.as_matrix)
display(sol)
# We can get a formula for lie algebra morphisms in this way:
L.as_matrix = L.as_matrix.subs(sol[0])
display(L.as_matrix)
# This method is not very smart and it does not work very well beyond these simple examples.

Matrix([
[L_0_0, L_0_1, L_0_2],
[L_1_0, L_1_1, L_1_2],
[L_2_0, L_2_1, L_2_2]])

[{L_0_2: 0, L_1_2: 0, L_2_2: L_0_0*L_1_1 - L_0_1*L_1_0}]

Matrix([
[L_0_0, L_0_1,                         0],
[L_1_0, L_1_1,                         0],
[L_2_0, L_2_1, L_0_0*L_1_1 - L_0_1*L_1_0]])

In [44]:
M, sol = heis.morphisms_to(W2,smbl='M')
M.as_matrix = M.as_matrix.subs(sol[0])
display(M.as_matrix)

Matrix([
[M_0_0, M_0_1, 0],
[M_1_0, M_1_1, 0],
[M_2_0, M_2_1, 0]])

In [46]:
K, sol = heis.morphisms_to(heis2,smbl='K')
K.as_matrix = K.as_matrix.subs(sol[0])
display(K.as_matrix)

Matrix([
[K_0_0, K_0_1,                                                     0],
[K_1_0, K_1_1,                                                     0],
[K_2_0, K_2_1,                                                     0],
[K_3_0, K_3_1,                                                     0],
[K_4_0, K_4_1, K_0_0*K_1_1 - K_0_1*K_1_0 + K_2_0*K_3_1 - K_2_1*K_3_0]])

### JET ALGEBRAS
$
\newcommand{\R}{\mathbb{R}}
\newcommand{\Jet}{\mathtt{Jet}}
\newcommand{\g}{\mathfrak{g}}
$

This part follows the paper
https://arxiv.org/abs/2201.04534

Given a stratified Lie algerba $\g$, 
we define jet spaces $\Jet^m(\g;\R)$
of order $m$ with values in $\R$.

The jet space $\Jet^m(\g;\R)$ is itself a Lie algebra.

In [49]:
# This snipped makes this part self-contained.
heis = LieAlgebra()
heis.dimension = 3
[X,Y,Z] = heis.basis_symbolic
heis.brackets.rules = {(X,Y):Z}
heis.declare_stratified(isit = True, growth_vector = [2,1], step = 2)

In [50]:
heisJet = JetAlgebra()
heisJet.name = 'Heisenberg Jet'
heisJet.lie_algebra_domain = heis
heisJet.order = 2
heisJet.build_me()

Step 1 of 7: Construct set of indices:
[((-1, 0, 0), 0), ((0, -1, 0), 0), ((2, 0, 0), @1), ((1, 1, 0), @1), ((0, 2, 0), @1), ((0, 0, 1), @1), ((0, 0, -1), 0), ((1, 0, 0), @1), ((0, 1, 0), @1), ((0, 0, 0), @1)]

Step 2 of 7: Construct HD basis:
{((0, 0, 0), @1): @1, ((1, 0, 0), @1): @b_0*@1, ((0, 1, 0), @1): @b_1*@1, ((2, 0, 0), @1): @b_0**2*@1, ((1, 1, 0), @1): @b_0*@b_1*@1 + @b_1*@b_0*@1, ((0, 2, 0), @1): @b_1**2*@1, ((0, 0, 1), @1): -@b_0*@b_1*@1}

Step 3 of 7: and outer basis:
[b_0, b_1, @b_0**2*@1, @b_0*@b_1*@1 + @b_1*@b_0*@1, @b_1**2*@1, -@b_0*@b_1*@1, b_2, @b_0*@1, @b_1*@1, @1]

Step 4 of 7: Construct sybolic basis:
[A^0_(-1, 0, 0), A^0_(0, -1, 0), A^@1_(2, 0, 0), A^@1_(1, 1, 0), A^@1_(0, 2, 0), A^@1_(0, 0, 1), A^0_(0, 0, -1), A^@1_(1, 0, 0), A^@1_(0, 1, 0), A^@1_(0, 0, 0)]

Step 5 of 7: Construct growth vector
[6, 3, 1]

Step 6 of 7: Construct functions from outer basis to the others.
Building transformation operations: this can take a lot of time. You may want to change the sou

In [55]:
# This snipped shows the description of the lie brackets in the jet space
# In the output, both the first column and the first row show the standard basis of the jet space
# Inside the matrix are the lie brackets of the corresponding elements in the basis.
basis = []
basis = heisJet.basis_symbolic
dim = heisJet.dimension
M = zeros(dim+1,dim+1)
for i in range(dim):
    M[i+1,0] = M[0,i+1] = basis[i]
for i in range(dim):
    for j in range(dim):
#         print(i,j)
        M[i+1,j+1] = heisJet(basis[i],basis[j])
display(M)

Matrix([
[             0,  A^0_(-1, 0, 0),  A^0_(0, -1, 0),  A^@1_(2, 0, 0),  A^@1_(1, 1, 0),  A^@1_(0, 2, 0),  A^@1_(0, 0, 1), A^0_(0, 0, -1),  A^@1_(1, 0, 0),  A^@1_(0, 1, 0), A^@1_(0, 0, 0)],
[A^0_(-1, 0, 0),               0,  A^0_(0, 0, -1), -A^@1_(1, 0, 0), -A^@1_(0, 1, 0),               0,               0,              0, -A^@1_(0, 0, 0),               0,              0],
[A^0_(0, -1, 0), -A^0_(0, 0, -1),               0,               0, -A^@1_(1, 0, 0), -A^@1_(0, 1, 0),  A^@1_(1, 0, 0),              0,               0, -A^@1_(0, 0, 0),              0],
[A^@1_(2, 0, 0),  A^@1_(1, 0, 0),               0,               0,               0,               0,               0,              0,               0,               0,              0],
[A^@1_(1, 1, 0),  A^@1_(0, 1, 0),  A^@1_(1, 0, 0),               0,               0,               0,               0,              0,               0,               0,              0],
[A^@1_(0, 2, 0),               0,  A^@1_(0, 1, 0),           

In [58]:
# same as above, but with a table.
from tabulate import tabulate
M_list = [[M[i,j] for j in range(M.cols)] for i in range(M.rows)]
#tabella = tabulate(M_list,headers="firstrow", tablefmt="html")
tabella = tabulate(M_list, tablefmt="html")
display(tabella)

0,"A^0_(-1, 0, 0)","A^0_(0, -1, 0)","A^@1_(2, 0, 0)","A^@1_(1, 1, 0)","A^@1_(0, 2, 0)","A^@1_(0, 0, 1)","A^0_(0, 0, -1)","A^@1_(1, 0, 0)","A^@1_(0, 1, 0)","A^@1_(0, 0, 0)"
"A^0_(-1, 0, 0)",0,"A^0_(0, 0, -1)","-A^@1_(1, 0, 0)","-A^@1_(0, 1, 0)",0,0,0,"-A^@1_(0, 0, 0)",0,0
"A^0_(0, -1, 0)","-A^0_(0, 0, -1)",0,0,"-A^@1_(1, 0, 0)","-A^@1_(0, 1, 0)","A^@1_(1, 0, 0)",0,0,"-A^@1_(0, 0, 0)",0
"A^@1_(2, 0, 0)","A^@1_(1, 0, 0)",0,0,0,0,0,0,0,0,0
"A^@1_(1, 1, 0)","A^@1_(0, 1, 0)","A^@1_(1, 0, 0)",0,0,0,0,0,0,0,0
"A^@1_(0, 2, 0)",0,"A^@1_(0, 1, 0)",0,0,0,0,0,0,0,0
"A^@1_(0, 0, 1)",0,"-A^@1_(1, 0, 0)",0,0,0,0,"A^@1_(0, 0, 0)",0,0,0
"A^0_(0, 0, -1)",0,0,0,0,0,"-A^@1_(0, 0, 0)",0,0,0,0
"A^@1_(1, 0, 0)","A^@1_(0, 0, 0)",0,0,0,0,0,0,0,0,0
"A^@1_(0, 1, 0)",0,"A^@1_(0, 0, 0)",0,0,0,0,0,0,0,0
"A^@1_(0, 0, 0)",0,0,0,0,0,0,0,0,0,0
